# 05_conditional_operations

Conditional Updates and Deletes

This example demonstrates bulk conditional operations:
- update_where() for conditional updates
- delete_where() for conditional deletes
- Working with pandas boolean masks
- Bulk data transformations

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import pandalchemy as pa

In [2]:
# Setup
engine = create_engine('sqlite:///:memory:')
db = pa.DataBase(engine)

In [3]:
print("Conditional Operations Example")

Conditional Operations Example


In [4]:
# Create sample employee data
employees_data = pd.DataFrame({
    'name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Eve', 'Frank', 'Grace', 'Henry'],
    'department': ['Engineering', 'Sales', 'Engineering', 'Marketing', 
                   'Sales', 'Engineering', 'Marketing', 'Sales'],
    'salary': [80000, 60000, 75000, 70000, 62000, 85000, 72000, 58000],
    'years': [5, 3, 4, 2, 3, 6, 2, 1],
    'performance': ['excellent', 'good', 'excellent', 'good', 
                    'average', 'excellent', 'good', 'average']
}, index=range(1, 9))

employees = db.create_table('employees', employees_data, primary_key='id')

### 1. Initial Employee Data

In [5]:
print(employees.to_pandas())

       name   department  salary  years performance
id                                                 
1     Alice  Engineering   80000      5   excellent
2       Bob        Sales   60000      3        good
3   Charlie  Engineering   75000      4   excellent
4     Diana    Marketing   70000      2        good
5       Eve        Sales   62000      3     average
6     Frank  Engineering   85000      6   excellent
7     Grace    Marketing   72000      2        good
8     Henry        Sales   58000      1     average


### 2. update_where() - Conditional Updates

In [6]:
# Simple condition update

### \n   a) Give 10% raise to excellent performers:

In [7]:
condition = employees._data['performance'] == 'excellent'
employees.update_where(condition, {
    'salary': lambda x: x * 1.10
})

print(f"✓ Updated {condition.sum()} employees")

# Multiple field update with dict

✓ Updated 3 employees


/Users/odosmatthews/Documents/coding/pandalchemy/src/pandalchemy/tracked_dataframe.py:1108: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[88000. 82500. 93500.]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  self._data.loc[condition, col] = self._data.loc[condition, col].apply(value)


### \n   b) Update Sales department info:

In [8]:
sales_condition = employees._data['department'] == 'Sales'
employees.update_where(sales_condition, {
    'department': 'Business Development'
})

print(f"      ✓ Renamed department for {sales_condition.sum()} employees")

employees.push()
print("\n   After updates:")
print(employees.to_pandas()[['name', 'department', 'salary', 'performance']])

      ✓ Renamed department for 3 employees

   After updates:
       name            department  salary performance
id                                                   
1     Alice           Engineering   88000   excellent
2       Bob  Business Development   60000        good
3   Charlie           Engineering   82500   excellent
4     Diana             Marketing   70000        good
5       Eve  Business Development   62000     average
6     Frank           Engineering   93500   excellent
7     Grace             Marketing   72000        good
8     Henry  Business Development   58000     average


### 3. Complex Conditions

In [9]:
# Multiple conditions with &, |

### a) Bonus for senior excellent performers

In [10]:
# First add bonus column with default
employees.add_column_with_default('bonus', 0)
employees.push()
employees.pull()

# Now update bonuses
senior_excellent = (employees._data['years'] >= 5) & \
                   (employees._data['performance'] == 'excellent')

employees.update_where(senior_excellent, {'bonus': 5000})

print(f"✓ {senior_excellent.sum()} employees get $5000 bonus")

# Salary-based condition

✓ 2 employees get $5000 bonus


### \n   b) Adjust low salaries:

In [11]:
low_salary = employees._data['salary'] < 65000
employees.update_where(low_salary, {
    'salary': lambda x: max(x * 1.05, 65000)
})

print(f"✓ Adjusted {low_salary.sum()} salaries to minimum $65,000")

employees.push()
print("\n   Current state:")
print(employees.to_pandas()[['name', 'salary', 'years', 'bonus']])

✓ Adjusted 3 salaries to minimum $65,000

   Current state:
       name  salary  years  bonus
id                               
1     Alice   88000      5   5000
2       Bob   65000      3      0
3   Charlie   82500      4      0
4     Diana   70000      2      0
5       Eve   65100      3      0
6     Frank   93500      6   5000
7     Grace   72000      2      0
8     Henry   65000      1      0


### 4. delete_where() - Conditional Deletes

In [12]:
# Create temp employee data
temp_data = pd.DataFrame({
    'name': ['Temp1', 'Temp2', 'Temp3'],
    'department': ['Engineering', 'Marketing', 'Engineering'],
    'salary': [45000, 42000, 48000],
    'years': [0.5, 0.3, 0.8],
    'performance': ['average', 'average', 'good'],
    'bonus': [0, 0, 0]
}, index=[20, 21, 22])

# Add temp employees
for idx, row in temp_data.iterrows():
    row_dict = row.to_dict()
    row_dict['id'] = idx  # Add the index as id
    employees.add_row(row_dict)

employees.push()

### \n   a) Added temporary employees:

In [13]:
print(f"      Total employees: {len(employees._data)}")

# Delete based on condition

      Total employees: 11


### \n   b) Remove employees with < 1 year experience:

In [14]:
deleted_count = employees.delete_where(employees._data['years'] < 1)

print(f"      ✓ Deleted {deleted_count} employees")

employees.push()
print(f"      Remaining employees: {len(employees._data)}")

      ✓ Deleted 3 employees
      Remaining employees: 8


### 5. Combining Conditions with Boolean Logic

In [15]:
# AND condition

### \n   a) Update high performers in Engineering:

In [16]:
condition_and = (employees._data['department'] == 'Engineering') & \
                (employees._data['performance'] == 'excellent')

employees.update_where(condition_and, {
    'bonus': lambda x: x + 3000
})
print(f"✓ {condition_and.sum()} engineers got extra $3000 bonus")

# OR condition

✓ 3 engineers got extra $3000 bonus


### \n   b) Flag employees needing review:

In [17]:
employees._data['needs_review'] = False
condition_or = (employees._data['performance'] == 'average') | \
               (employees._data['years'] < 2)

employees.update_where(condition_or, {'needs_review': True})
print(f"      ✓ {condition_or.sum()} employees flagged for review")

# NOT condition

      ✓ 2 employees flagged for review


### \n   c) Update everyone except Marketing:

In [18]:
not_marketing = ~(employees._data['department'] == 'Marketing')
employees.update_where(not_marketing, {
    'salary': lambda x: x + 1000  # $1000 across-the-board raise
})
print(f"✓ {not_marketing.sum()} employees got $1000 raise")

employees.push()

✓ 6 employees got $1000 raise


### 6. Real-World Example: Customer Status Updates

In [19]:
customers_data = pd.DataFrame({
    'name': ['Acme Corp', 'TechStart', 'DataCo', 'SmallBiz', 'MegaCorp'],
    'revenue': [500000, 150000, 800000, 50000, 1200000],
    'signup_date': ['2023-01-15', '2023-06-20', '2022-12-01', 
                    '2024-01-10', '2022-08-15'],
    'status': ['active', 'active', 'active', 'trial', 'active'],
    'support_tier': ['standard', 'standard', 'standard', 'basic', 'standard']
}, index=range(1, 6))

customers = db.create_table('customers', customers_data, primary_key='id')

print("\n   Initial customers:")
print(customers.to_pandas())

# Upgrade high-value customers


   Initial customers:
         name  revenue signup_date  status support_tier
id                                                     
1   Acme Corp   500000  2023-01-15  active     standard
2   TechStart   150000  2023-06-20  active     standard
3      DataCo   800000  2022-12-01  active     standard
4    SmallBiz    50000  2024-01-10   trial        basic
5    MegaCorp  1200000  2022-08-15  active     standard


### \n   a) Upgrade high-revenue customers to premium:

In [20]:
high_value = customers._data['revenue'] > 500000
customers.update_where(high_value, {'support_tier': 'premium'})
print(f"      ✓ {high_value.sum()} customers upgraded")

# Convert trials to active

      ✓ 2 customers upgraded


### \n   b) Convert trial to paid after certain revenue:

In [21]:
trial_conversion = (customers._data['status'] == 'trial') & \
                   (customers._data['revenue'] > 40000)
customers.update_where(trial_conversion, {'status': 'active'})
print(f"      ✓ {trial_conversion.sum()} trials converted")

# Remove inactive small accounts

      ✓ 1 trials converted


### \n   c) Remove low-value inactive accounts:

In [22]:
to_remove = (customers._data['revenue'] < 100000) & \
            (customers._data['status'] != 'active')
removed = customers.delete_where(to_remove)
print(f"      ✓ {removed} accounts removed")

customers.push()
print("\n   Final customers:")
print(customers.to_pandas())

      ✓ 0 accounts removed

   Final customers:
         name  revenue signup_date  status support_tier
id                                                     
1   Acme Corp   500000  2023-01-15  active     standard
2   TechStart   150000  2023-06-20  active     standard
3      DataCo   800000  2022-12-01  active      premium
4    SmallBiz    50000  2024-01-10  active        basic
5    MegaCorp  1200000  2022-08-15  active      premium


### 7. Batch Updates with Calculations

In [23]:
# Price adjustment example
products_data = pd.DataFrame({
    'name': ['Widget', 'Gadget', 'Doohickey', 'Thingamajig', 'Whatsit'],
    'price': [9.99, 19.99, 29.99, 39.99, 14.99],
    'category': ['A', 'B', 'A', 'C', 'B'],
    'stock': [100, 50, 75, 25, 200]
}, index=range(1, 6))

products = db.create_table('products', products_data, primary_key='id')

print("\n   Initial products:")
print(products.to_pandas())

# Discount low-stock items


   Initial products:
           name  price category  stock
id                                    
1        Widget   9.99        A    100
2        Gadget  19.99        B     50
3     Doohickey  29.99        A     75
4   Thingamajig  39.99        C     25
5       Whatsit  14.99        B    200


### \n   a) 10% discount on low-stock items:

In [24]:
low_stock = products._data['stock'] < 50
products.update_where(low_stock, {
    'price': lambda x: round(x * 0.9, 2)
})

# Category-based pricing

### \n   b) Premium pricing for category C:

In [25]:
cat_c = products._data['category'] == 'C'
products.update_where(cat_c, {
    'price': lambda x: round(x * 1.15, 2)
})

products.push()
print("\n   After pricing updates:")
print(products.to_pandas())


   After pricing updates:
           name  price category  stock
id                                    
1        Widget   9.99        A    100
2        Gadget  19.99        B     50
3     Doohickey  29.99        A     75
4   Thingamajig  41.39        C     25
5       Whatsit  14.99        B    200


### 8. Performance Tip: Single Update vs Loop

In [26]:
print("\n   ❌ SLOW - Loop with individual updates:")
print("      for idx in df[df['age'] > 65].index:")
print("          table.update_row(idx, {'senior': True})")
print()
print("   ✓ FAST - Single conditional update:")
print("      table.update_where(df['age'] > 65, {'senior': True})")
print()
print("   Difference: One SQL UPDATE vs many individual UPDATEs")

print("\n" + "=" * 70)
print("Example Complete!")
print("Key Takeaways:")
print("  • update_where() for bulk conditional updates")
print("  • delete_where() for bulk conditional deletes")
print("  • Use pandas boolean masks for conditions")
print("  • Combine with & (and), | (or), ~ (not)")
print("  • Much faster than looping with update_row()")


   ❌ SLOW - Loop with individual updates:
      for idx in df[df['age'] > 65].index:
          table.update_row(idx, {'senior': True})

   ✓ FAST - Single conditional update:
      table.update_where(df['age'] > 65, {'senior': True})

   Difference: One SQL UPDATE vs many individual UPDATEs

Example Complete!
Key Takeaways:
  • update_where() for bulk conditional updates
  • delete_where() for bulk conditional deletes
  • Use pandas boolean masks for conditions
  • Combine with & (and), | (or), ~ (not)
  • Much faster than looping with update_row()
